In [16]:
#Library installations
import numpy as np

from qiskit import QuantumCircuit, transpile
from qiskit.visualization.counts_visualization import plot_histogram
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qbraid.runtime.qiskit import QiskitRuntimeProvider
from qbraid.runtime.qiskit.job import QiskitJob

from qiskit_ibm_runtime import QiskitRuntimeService

from dotenv import load_dotenv

import os

In [17]:
#Building a 100 qubit quantum circuit with all qubits in equal superposition using hadamard gate
circ = QuantumCircuit(100)
for i in range(100):
    circ.h(i)
circ.measure_all()

In [ ]:
#Visualization of circuit
circ.draw()

In [18]:
def launch_job(device_name, shot_count):
    load_dotenv()
    API_KEY = os.getenv('IBM_APIKEY')
    provider = QiskitRuntimeProvider(API_KEY)
    device = provider.get_device(device_name)
    pm = generate_preset_pass_manager(optimization_level=1, target=device._backend.target)
    transpiled_circuit = pm.run(circ)
    job = device.run(transpiled_circuit, shots=shot_count, memory=True)
    print(job._job_id)


def get_job_results(job_id):
    API_KEY = os.getenv('IBM_APIKEY')
    provider = QiskitRuntimeService('ibm_quantum', API_KEY)
    job = provider.job(job_id)
    QBraidJob = QiskitJob(job_id, job)
    result = QBraidJob.result()

    rawData = result.measurements()

    unbrokenData = []
    for shot in rawData:
        for meas in shot:
            unbrokenData.append(meas)

    data = []
    for i in range(0, len(unbrokenData), 100):
        data.append(''.join(map(str, unbrokenData[i:i+100])))
    
    return data

def write_data_to_file(data, file_path, labels):
    with open(file_path, 'a') as file:
        labelString = ''
        for label in labels:
            labelString += (label + ' ')
        labelString = labelString[:-1]
        for output in data:
            file.write(output + ' ' + labelString + '\n')
    file.close()

In [19]:
launch_job('ibm_brisbane', 10000)

ctmnhb54gjh0008pcke0


In [21]:
launch_job('ibm_kyoto', 10000)

ctmnhr7sj17g0081x3zg


In [22]:
launch_job('ibm_osaka', 10000)

ctmnhwfsj17g0081x400


In [23]:
launch_job('ibm_sherbrooke', 10000)

ctmnhyzsj17g0081x40g


In [24]:
write_data_to_file(get_job_results('ctmnhb54gjh0008pcke0'), 'data/IBM_Superposition_QRNG_100qubit.txt', ['quantum', 'ibm_brisbane'])

In [25]:
write_data_to_file(get_job_results('ctmnhr7sj17g0081x3zg'), 'data/IBM_Superposition_QRNG_100qubit.txt', ['quantum', 'ibm_kyoto'])

In [26]:
write_data_to_file(get_job_results('ctmnhwfsj17g0081x400'), 'data/IBM_Superposition_QRNG_100qubit.txt', ['quantum', 'ibm_osaka'])

In [27]:
write_data_to_file(get_job_results('ctmnhyzsj17g0081x40g'), 'data/IBM_Superposition_QRNG_100qubit.txt', ['quantum', 'ibm_sherbrooke'])

In [ ]:
#Creating euql amount of non-quantum training data using numpy random module
with open('data/IBM_Superposition_QRNG_100qubit.txt', 'a') as file:
    file.write('\n')
    rng = np.random.default_rng()
    for i in range(40000):
        prng = rng.integers(0, 2, 100)
        rngString = ''.join(str(x) for x in prng)
        output = rngString + ' non-quantum N/A\n'
        file.write(output)
    
file.close()

Refactoring QRNG generation code for 10 qubit circuit